In [1]:
import cv2
import glob
import numpy as np
from queue import PriorityQueue

# show plot first run
%matplotlib inline 
# test autocompletion with tab or tab+shift
%config IPCompleter.greedy=True 
############################################################
#
#              Simple Image Retrieval
#
############################################################


# implement distance function
def distance(a, b):
    return np.linalg.norm(a-b)

def create_keypoints(w, h):
    keypoints = []
    keypointSize = 29
    numkeypoints = 15
    
    offset = int(keypointSize/2)
    p_h = np.linspace(offset,h-offset, numkeypoints)
    p_w = np.linspace(offset,w-offset, numkeypoints)
    point_h=p_h.astype(int)
    point_w=p_w.astype(int)
    
    #print(point_h)
    #print(point_w)
    
    # YOUR CODE HERE
    for r in point_h:
        for c in point_w:
            #print(r,c)
            keypoints.append(cv2.KeyPoint(r,h, keypointSize))
    
    return keypoints



In [2]:
# 1. preprocessing and load
images = glob.glob('./images/db/*/*/*.jpg')
test_images = glob.glob('./images/db/*/*.jpg')
# 2. create keypoints on a regular grid (cv2.KeyPoint(r, c, keypointSize), as keypoint size use e.g. 11)
keypoints = create_keypoints(255, 255)
#print(len(keypoints))

In [3]:
# 3. use the keypoints for each image and compute SIFT descriptors
#    for each keypoint. this compute one descriptor for each image.
# YOUR CODE HERE

img_set_train = []
descriptors = []
sift = cv2.xfeatures2d.SIFT_create()
for i,name in enumerate(images):
    img_set_train.append(cv2.imread(name))

    #cv2.imshow("mog", img)
    #cv2.waitKey(0) 
    key,des = sift.compute(img_set_train[i],keypoints)
    descriptors.append(des)
    
    '''
    #for i in range(len(keypoints)):
    a1=np.array(keypoints[0])
    a2=np.array(key[0])
    print(np.array_equal(a1,a2))
    print('=',(a1==a2).all())
    print(keypoints[0].angle,key[0].angle)
    print(keypoints[0].class_id,key[0].class_id)
    print(keypoints[0].octave,key[0].octave)
    print(keypoints[0].pt[0],key[0].pt[0])
    print(keypoints[0].pt[1],key[0].pt[1])
    print(keypoints[0].response,key[0].response)
    print(keypoints[0].size,key[0].size)
    '''

print('number of images:',len(descriptors))
print('pro Bild je keypoint:',len(descriptors[0]))
print('pro keypoint number data:',len(descriptors[0][0]))
print('description matrix per image:',descriptors[0].shape)

number of images: 20
pro Bild je keypoint: 225
pro keypoint number data: 128
description matrix per image: (225, 128)


In [7]:
# 4. use one of the query input image to query the 'image database' that
#    now compress to a single area. Therefore extract the descriptor and
#    compare the descriptor to each image in the database using the L2-norm
#    and save the result into a priority queue (q = PriorityQueue())

# YOUR CODE HERE
testimgsize = 200
trainimgsize = int(testimgsize/2)

Img_mutires = np.zeros((testimgsize, 10+testimgsize+trainimgsize*10, 3), np.uint8)

for i,timg in enumerate(test_images):
    q = PriorityQueue()
    
    test_img = cv2.imread(timg)
 
    #cv2.imshow("mog", img)
    #cv2.waitKey(0) 
    
    key,des = sift.compute(test_img,keypoints)
    for j,descrtrain in enumerate(descriptors):
        dist = distance(des,descrtrain)
        q.put((dist, img_set_train[j]))

    smalltimg = cv2.resize(test_img,(testimgsize,testimgsize)) 
    Img_mutires[0:testimgsize,0:testimgsize] = smalltimg
    
    # 5. output (save and/or display) the query results in the order of smallest distance=
    count = 0
    while not q.empty():
        retr = q.get()[1]
        smallqimg = cv2.resize(retr,(trainimgsize,trainimgsize))
        row = int(count/10)
        col = count%10
        Img_mutires[trainimgsize*row:trainimgsize*(row+1),
            testimgsize+10+ trainimgsize*col : testimgsize+10+ trainimgsize*(col+1)] = smallqimg 
        count+=1
        
        #cv2.imshow("query", newimg)
        #cv2.waitKey(0) 
    
    cv2.imwrite('Query_'+str(i)+'.png',Img_mutires)

    cv2.destroyAllWindows()